In [1]:
import tensorflow as tf

import numpy as np
import os
import time

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [2]:
# Read, then decode for py2 compat.
with open(path_to_file, 'rb') as reader:
  text=reader.read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

65 unique characters


In [5]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [6]:
# The maximum length sentence we want for a single input in characters
# 比如说全文有101个词，那实际上就只有一个样本，即1~100个字去预测第2~101个
# 但如果有202个，那就是俩样本，1~100->2~101 101~201->102~202
# 因为是rnn嘛
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) #

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [7]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [8]:
# 做截断 101个字符成为一个batch
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000
# 这个batch是分batch了
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset



<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),# 这个None指的是每个序列的长度
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
  emb=tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None])(input_example_batch)
  print(emb.shape)
  gru=tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(emb)
  print(gru.shape) # 可以看到gur这东西接受的是batch,sequence,emb;吐出来的是batch,sequence,rnn_units
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# 最后这个dense层接受了一个64，100，1024;然后对每一组1024做dense，输出了64，100，65

(64, 100, 256)
(64, 100, 1024)
(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [17]:
print(example_batch_predictions[0])
# 按照概率来选择一个结果，example_batch_predictions形如100,65；从每一串65里按照logits作为概率抽取一个样本
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices)


tf.Tensor(
[[ 5.1035383e-04 -1.4993914e-03 -3.4585362e-05 ... -6.4427638e-03
  -1.0574569e-03  1.5593479e-02]
 [-4.6918606e-03  2.3096479e-03 -8.7016486e-03 ... -1.3214234e-02
   3.6713202e-04  5.2284924e-03]
 [-6.5695890e-04 -4.1443401e-04  2.7617340e-03 ...  1.5086471e-03
   7.9773320e-03 -5.1984685e-03]
 ...
 [ 4.2674481e-03 -5.2045817e-03  9.2306584e-03 ...  1.8277504e-03
   8.5354215e-03 -2.1728957e-03]
 [ 9.7058024e-03 -5.7274955e-03  6.3430849e-03 ...  6.2243929e-03
   1.1353554e-02 -2.6341486e-03]
 [ 1.8465169e-02  3.6756601e-04 -6.5496247e-03 ...  1.2467973e-02
   6.2542506e-03 -3.9660567e-03]], shape=(100, 65), dtype=float32)
tf.Tensor(
[[26]
 [29]
 [27]
 [27]
 [57]
 [55]
 [39]
 [ 9]
 [64]
 [64]
 [15]
 [36]
 [55]
 [17]
 [43]
 [21]
 [ 2]
 [64]
 [54]
 [52]
 [28]
 [42]
 [30]
 [55]
 [ 7]
 [23]
 [ 7]
 [42]
 [57]
 [50]
 [19]
 [ 6]
 [11]
 [14]
 [15]
 [29]
 [36]
 [51]
 [51]
 [60]
 [53]
 [50]
 [44]
 [46]
 [23]
 [53]
 [50]
 [22]
 [38]
 [40]
 [20]
 [26]
 [51]
 [31]
 [ 4]
 [48]
 [15]
 [3

In [18]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([26, 29, 27, 27, 57, 55, 39,  9, 64, 64, 15, 36, 55, 17, 43, 21,  2,
       64, 54, 52, 28, 42, 30, 55,  7, 23,  7, 42, 57, 50, 19,  6, 11, 14,
       15, 29, 36, 51, 51, 60, 53, 50, 44, 46, 23, 53, 50, 22, 38, 40, 20,
       26, 51, 31,  4, 48, 15, 36, 52, 35, 37, 45, 12, 14, 44, 46,  0,  2,
       25,  2, 52, 15, 37, 30, 30, 57, 55, 10, 49, 26,  1, 31,  5, 51, 52,
        1, 63, 57,  8, 57, 14, 61, 46, 30,  8, 46, 13, 18, 51, 27])

In [19]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 " God hath made for himself to\nmar.\n\nNurse:\nBy my troth, it is well said; 'for himself to mar,'\nquoth"

Next Char Predictions: 
 "NQOOsqa3zzCXqEeI!zpnPdRq-K-dslG,;BCQXmmvolfhKolJZbHNmS&jCXnWYg?Bfh\n!M!nCYRRsq:kN S'mn ys.sBwhR.hAFmO"


In [20]:
print(example_batch_predictions.shape)
print(target_example_batch.shape)

(64, 100, 65)
(64, 100)


In [21]:
def loss(labels, logits):
  # 这个函数里的logits的最后两个维度去对labes的最后一个维度
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())
model.compile(optimizer='adam', loss=loss)

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174996


In [107]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=10

In [82]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
 68/172 [==========>...................] - ETA: 2:07 - loss: 3.2222

KeyboardInterrupt: 

In [22]:
model.build()


In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
# build用来构建模型，因为load_weight应该是个lazy mode
# model.build(tf.TensorShape([1, None]))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 10

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)
    print(predictions.shape)
    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    # 只取最后一个字
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    print(predicted_id.shape)
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    print(input_eval.shape) # 从0d扩充到[1,1]
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"ROMEO: "))

(7, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
(1, 65)
()
(1, 1)
ROMEO:  :-hcEG.L!


In [26]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [27]:
optimizer = tf.keras.optimizers.Adam()

In [31]:
@tf.function
def train_step(inp, target):
  print(inp) # 一个batch的input
  print(target) # 一个batch的target
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [32]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))
    break
  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

tf.Tensor(
[[53 51 43 ... 15 53 51]
 [ 1 57 59 ... 53 44  1]
 [43 56 60 ... 50 50  1]
 ...
 [47 57  1 ... 46 39 58]
 [ 1 41 53 ... 30 10  0]
 [ 1 58 46 ... 47 58  0]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[51 43 58 ... 53 51 43]
 [57 59 47 ... 44  1 46]
 [56 60 63 ... 50  1 39]
 ...
 [57  1 58 ... 39 58  1]
 [41 53 56 ... 10  0 33]
 [58 46 43 ... 58  0 37]], shape=(64, 100), dtype=int64)
Epoch 1 Batch 0 Loss 3.575183629989624
Epoch 1 Loss 3.5752
Time taken for 1 epoch 2.2520713806152344 sec

tf.Tensor(
[[53 51 54 ... 63  1 58]
 [53 52  1 ... 53 59 50]
 [57 43 52 ... 58  8  1]
 ...
 [56 43 58 ... 47 45 46]
 [ 1 51 47 ... 39 56  1]
 [ 6  1 57 ...  1 58 46]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[51 54 50 ...  1 58 53]
 [52  1 53 ... 59 50 42]
 [43 52 58 ...  8  1 20]
 ...
 [43 58 46 ... 45 46 58]
 [51 47 52 ... 56  1 40]
 [ 1 57 53 ... 58 46 47]], shape=(64, 100), dtype=int64)
Epoch 2 Batch 0 Loss 3.793905735015869
Epoch 2 Loss 3.7939
Time taken for 1 epoch 2.230330467224121 se

NameError: name 'checkpoint_prefix' is not defined

In [28]:
for (batch_n, (inp, target)) in enumerate(dataset):
  print(batch_n)
  print(inp)
  print(target)

0
tf.Tensor(
[[17 50 43 ...  0 21 58]
 [ 1 43 60 ... 56 59 52]
 [15 39 58 ... 20 59 45]
 ...
 [42 43 52 ... 11  1 51]
 [43 56 57 ... 58 46 43]
 [10  0 26 ... 57  1 58]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[50 43 60 ... 21 58  1]
 [43 60 47 ... 59 52 52]
 [39 58 50 ... 59 45 46]
 ...
 [43 52 10 ...  1 51 39]
 [56 57 53 ... 46 43 63]
 [ 0 26 53 ...  1 58 46]], shape=(64, 100), dtype=int64)
1
tf.Tensor(
[[59 57 43 ... 50 57  2]
 [52 42  1 ... 43 47 56]
 [ 0 13 52 ...  1 58 56]
 ...
 [46 39 42 ... 40 43 56]
 [52 57  8 ... 56  1 52]
 [ 1 58 46 ...  1 49 52]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[57 43 42 ... 57  2  0]
 [42  1 45 ... 47 56  1]
 [13 52 42 ... 58 56 59]
 ...
 [39 42  1 ... 43 56 58]
 [57  8  1 ...  1 52 43]
 [58 46 43 ... 49 52 53]], shape=(64, 100), dtype=int64)
2
tf.Tensor(
[[ 1 58 46 ... 46 39 58]
 [57  1 57 ...  1 54 50]
 [57 58 43 ... 10  0 21]
 ...
 [43 58 43 ... 42  1 52]
 [ 0 26 53 ... 27 26 32]
 [ 6  0 35 ...  1 40 39]], shape=(64, 100), dtype=int64)
tf.T

[<tf.Tensor 'while/add:0' shape=() dtype=int32>]

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4], dtype=int32)>